In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from anchor.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = 'figures'

import flotilla
study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', 
                        flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')
# study = flotilla.embark('singlecell_pnm_figure1_supplementary_post_splicing_filtering')

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-05-13 09:46:38	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-05-13 09:46:38	Parsing datapackage to create a Study object
2016-05-13 09:47:15	Initializing Study
2016-05-13 09:47:15	Initializing Predictor configuration manager for Study
2016-05-13 09:47:15	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-05-13 09:47:15	Added ExtraTreesClassifier to default predictors
2016-05-13 09:47:15	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-05-13 09:47:15	Added ExtraTreesRegressor to default predictors
2016-05-13 09:47:15	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-05-13 09:47:15	Added GradientBoostingClassifier to default predictors
2016-05-13 09:47:15	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [11]:
study.splicing.minimum_samples

10

In [12]:
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=study.splicing.minimum_samples))

In [13]:
psi_filtered.head()

,exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,exon:chr10:101507014-101507147:+@exon:chr10:101514286-101514391:+@exon:chr10:101515382-101515465:+,exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-@exon:chr10:101995424-101995521:-,exon:chr10:102003455-102003534:-@exon:chr10:101997779-101997988:-@exon:chr10:101996607-101996726:-,exon:chr10:102013178-102013296:-@exon:chr10:102010005-102010089:-@exon:chr10:102006552-102006692:-,exon:chr10:102019975-102020076:-@exon:chr10:102016019-102016233:-@exon:chr10:102013178-102013296:-,...,exon:chrY:15560897-15560946:-@exon:chrY:15526615-15526673:-@exon:chrY:15522873-15522993:-,exon:chrY:15591134-15591384:-@exon:chrY:15582001-15582109:-@exon:chrY:15560897-15560946:-,exon:chrY:21903204-21903374:-@exon:chrY:21901414-21901548:-@exon:chrY:21897507-21897636:-,exon:chrY:21903621-21903743:-@exon:chrY:21903204-21903374:-@exon:chrY:21901414-21901548:-,exon:chrY:22744476-22744579:+@exon:chrY:22746360-22746410:+@exon:chrY:22749910-22749991:+,exon:chrY:2709527-2709668:+@exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+,exon:chrY:2710206-2710283:+@exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+,exon:chrY:2712118-2712298:+@exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+,exon:chrY:2713687-2713784:+@exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+,exon:chrY:2722641-2722812:+@exon:chrY:2733129-2733286:+@exon:chrY:2734834-2734903:+
CVN_01,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1
CVN_02,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
CVN_03,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
CVN_04,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1
CVN_05,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,1,1,1,1,1


## Calculate *voyage* space via `bon voyage`

In [14]:
from astrolabe import Waypoints

In [15]:
ws = Waypoints()

waypoints = psi_filtered.groupby(study.sample_id_to_phenotype).apply(
    lambda x: ws.fit_transform(x.dropna(how='all', axis=1)))

In [16]:
waypoints.max()

0    1
1    1
dtype: float64

In [17]:
pd.DataFrame(ws.seed_data_transformed).max()

0    0.840884
1    0.999992
dtype: float64

In [18]:
transitions = study.phenotype_transitions + [('iPSC', 'MN')]
transitions

[(u'iPSC', u'NPC'), (u'NPC', u'MN'), ('iPSC', 'MN')]

In [19]:
from astrolabe.voyages import Voyages

In [20]:
v = Voyages()

voyages = v.voyages(waypoints, transitions)
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,exon:chr10:101165513-101165617:-@exon:chr10:10...,0.000000,0.034483,0.034483,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,exon:chr10:101480744-101480825:-@exon:chr10:10...,0.157895,-0.028862,0.160511,iPSC,NPC,$\searrow$,iPSC-NPC
2,exon:chr10:101507014-101507147:+@exon:chr10:10...,0.260684,-0.034188,0.262916,iPSC,NPC,$\searrow$,iPSC-NPC
3,exon:chr10:101997779-101997988:-@exon:chr10:10...,0.181818,-0.148485,0.234746,iPSC,NPC,$\searrow$,iPSC-NPC
4,exon:chr10:102019975-102020076:-@exon:chr10:10...,-0.107143,0.285714,0.305143,iPSC,NPC,$\nwarrow$,iPSC-NPC


In [21]:
voyages['transition'] = voyages.group1 + '-' + voyages.group2
voyages.head()

,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition
0,exon:chr10:101165513-101165617:-@exon:chr10:10...,0.000000,0.034483,0.034483,iPSC,NPC,$\nwarrow$,iPSC-NPC
1,exon:chr10:101480744-101480825:-@exon:chr10:10...,0.157895,-0.028862,0.160511,iPSC,NPC,$\searrow$,iPSC-NPC
2,exon:chr10:101507014-101507147:+@exon:chr10:10...,0.260684,-0.034188,0.262916,iPSC,NPC,$\searrow$,iPSC-NPC
3,exon:chr10:101997779-101997988:-@exon:chr10:10...,0.181818,-0.148485,0.234746,iPSC,NPC,$\searrow$,iPSC-NPC
4,exon:chr10:102019975-102020076:-@exon:chr10:10...,-0.107143,0.285714,0.305143,iPSC,NPC,$\nwarrow$,iPSC-NPC


In [22]:
voyages.shape

(17814, 8)

### Add group1 and group2 modalities

In [23]:
study.supplemental.modalities_tidy.head()

,phenotype,event_id,modality
0,MN,exon:chr10:101165513-101165617:-@exon:chr10:10...,included
1,MN,exon:chr10:101480744-101480825:-@exon:chr10:10...,bimodal
2,MN,exon:chr10:102114184-102114389:+@exon:chr10:10...,included
3,MN,exon:chr10:102286732-102286831:-@exon:chr10:10...,included
4,MN,exon:chr10:102747070-102747240:-@exon:chr10:10...,bimodal


In [26]:
groups = 'group1', 'group2'
voyages_modalities = voyages.copy()

for group in groups:
    voyages_modalities = voyages_modalities.merge(study.supplemental.modalities_tidy, 
                                       left_on=[group, 'event_id'], 
                                       right_on=['phenotype', 'event_id'], copy=False)
    print(voyages_modalities.shape)
    voyages_modalities = voyages_modalities.drop(['phenotype'], axis=1)
    voyages_modalities = voyages_modalities.rename(columns={'modality': '{}_modality'.format(group)})
print(voyages_modalities.shape)
voyages_modalities.head()

(17814, 10)
(17814, 11)
(17814, 10)


,event_id,$\Delta x$,$\Delta y$,magnitude,group1,group2,direction,transition,group1_modality,group2_modality
0,exon:chr10:101165513-101165617:-@exon:chr10:10...,0.000000,0.034483,0.034483,iPSC,NPC,$\nwarrow$,iPSC-NPC,included,included
1,exon:chr10:101165513-101165617:-@exon:chr10:10...,0.000000,0.034483,0.034483,iPSC,MN,$\nwarrow$,iPSC-MN,included,included
2,exon:chr10:101165513-101165617:-@exon:chr10:10...,0.000000,0.000000,0.000000,NPC,MN,NaN,NPC-MN,included,included
3,exon:chr10:101480744-101480825:-@exon:chr10:10...,0.157895,-0.028862,0.160511,iPSC,NPC,$\searrow$,iPSC-NPC,included,bimodal
4,exon:chr10:101480744-101480825:-@exon:chr10:10...,0.058824,0.070209,0.091594,iPSC,MN,$\nearrow$,iPSC-MN,included,bimodal


## Add the data to the study and save it

In [27]:
study.supplemental.waypoints = waypoints
study.supplemental.voyages = voyages_modalities

In [28]:
study.save('singlecell_pnm_figure4_voyages', flotilla_dir='/projects/ps-yeolab/obotvinnik/flotilla_projects/')

Wrote datapackage to /projects/ps-yeolab/obotvinnik/flotilla_projects//singlecell_pnm_figure4_voyages/datapackage.json
